In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [17]:
import tensorflow

Adding Natural language toolkit

In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

Using a dataset from kaggle

In [19]:
df = pd.read_csv('yelp.csv')

In [20]:
df.columns

Index(['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id',
       'cool', 'useful', 'funny'],
      dtype='object')

# Assumptions #

for this we can assume that number of stars strictly depends upon the key words of joy as sadness 

In [21]:
df.drop('business_id',axis=1,inplace=True)

In [22]:
df.drop('review_id',axis=1,inplace = True)

In [23]:
df.drop('type',axis=1,inplace=True)

In [24]:
df.drop('user_id',axis=1,inplace=True)

In [25]:
df

,date,stars,text,cool,useful,funny
0,2011-01-26,5,My wife took me here on my birthday for breakf...,2,5,0
1,2011-07-27,5,I have no idea why some people give bad review...,0,0,0
2,2012-06-14,4,love the gyro plate. Rice is so good and I als...,0,1,0
3,2010-05-27,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",1,2,0
4,2012-01-05,5,General Manager Scott Petello is a good egg!!!...,0,0,0
...,...,...,...,...,...,...
9995,2012-07-28,3,First visit...Had lunch here today - used my G...,1,2,0
9996,2012-01-18,4,Should be called house of deliciousness!\n\nI ...,0,0,0
9997,2010-11-16,4,I recently visited Olive and Ivy for business ...,0,0,0
9998,2012-12-02,2,My nephew just moved to Scottsdale recently so...,0,0,0


In [26]:
df2 = df.drop(['cool','date','useful','funny'],axis=1)

In [27]:
df2['stars'].unique()

array([5, 4, 2, 3, 1], dtype=int64)

In [28]:
df2

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,5,General Manager Scott Petello is a good egg!!!...
...,...,...
9995,3,First visit...Had lunch here today - used my G...
9996,4,Should be called house of deliciousness!\n\nI ...
9997,4,I recently visited Olive and Ivy for business ...
9998,2,My nephew just moved to Scottsdale recently so...


In [29]:
for i in range(len(df2['stars'])):
    if(df2['stars'][i] == np.nan):
        print('Lauda')

In [30]:
ps = PorterStemmer()
corpus=[]

There is no missing value for the target column

In [15]:
ps = PorterStemmer()
corpus=[]

for i in range(len(df2['text'])):
    
    rev = re.sub("^[a-zA-Z]",' ', df2['text'][i]) 
    rev = rev.lower()
    rev = rev.split()
    rev = [ps.stem(word) for word in rev if set(stopwords.words())]
    rev = ' '.join(rev)
    corpus.append(rev)
    
    print(i,end='\r')
print()

9999


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000)


In [34]:
y = df2['stars']

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

NameError: name 'X' is not defined

In [20]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [21]:

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])


In [22]:
search_spaces = {
    'vect__ngram_range': Categorical([(1, 1), (1, 2)]),  # Unigrams or bigrams
    'tfidf__use_idf': Categorical([True, False]),
    'clf__alpha': Real(0.1, 1.0, prior='log-uniform'),  # Smoothing parameter
}


In [24]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
y_train

7389    5
9275    4
2995    1
5316    5
356     4
       ..
9225    3
4859    5
3264    4
9845    3
2732    5
Name: stars, Length: 8000, dtype: int64

In [26]:
model = MultinomialNB()

In [27]:
model.fit(X_train,y_train)

MultinomialNB()

In [28]:
pred  = model.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score

In [30]:
accuracy_score(y_test,pred)

0.509

using Basysian optimization for hyperparameters

In [ ]:
bayes_search = BayesSearchCV(
    estimator=text_clf,
    search_spaces=search_spaces,
    n_iter=10,
    cv=5,
    n_jobs=-1,
    random_state=42,
    scoring='accuracy'
)
bayes_search.fit(X_train, y_train)

In [ ]:
print("Best parameters:", bayes_search.best_params_)

In [ ]:
predicted = bayes_search.best_estimator_.predict(X_test)
accuracy = np.mean(predicted == y_test)
print("Accuracy:", accuracy)

In [14]:
def test_model(text):
    stars = {
    '1 stars' : 1,
    '2 stars' : 2,
    '3 stars' : 3,
    '4 stars' : 4,
    '5 stars' : 5,
    }
    ps = PorterStemmer()
    corpus=[]
    rev = re.sub('^[a-zA-Z]',' ',text)
    rev = rev.lower()
    rev = rev.split()
    rev = [ps.stem(word) for word in rev if word not in set(stopwords.words())]
    rev = ' '.join(rev)
    
    rev = cv.transform([rev]).toarray()
    
    output = model.predict(rev)[0]
    
    keys = list(stars)
    values = list(stars.values())
    position = values.index(output)
    
    output = keys[position]
    
    print(output)

In [5]:
import joblib


In [10]:
model = joblib.load('stars.sav')

In [36]:
#test_model('There is a waitlist but it goes quite fast. They expanded the restaurant and it was quite nosy in there.. food is still good an other than the peppers they need another vege app option. The ramen is good and they can hard boil Those eggs for you! Servers were nice Everything came fast Felt rushed to leave because of people Waiting')

In [33]:
test_model('Waterbar is a perfect place for work lunches, dinners, and other celebratory meals, and that is exactly what we did. My team and I went here for lunch, and we had such a great time. Their food was incredibly delicious! I ordered the vegetarian ravioli, and it was very tasty. I believe the filling was some sort of squash(?), but whatever it was, it was very creamy. The creamy texture of the ravioli was a perfect pairing with the crunchy texture of the seeds. It was a great mix of crunch and silkiness in every bite, which made me want more. I also ordered the Petrale Sole, which was also a star on their menu. At first glance, it seemed underwhelming, but the flavor brought out its stardom. I am never a fan of carrots, but every bite of the carrot was dreamy. The fish itself was very well-seasoned and paired well with the vadouvan and Greek yogurt. On top of the delicious food, the customer service was fantastic! Before going here, we were reviewing the online menu and were each looking forward to specific dishes - mainly the rockfish and chips and baked oysters. However, the online menu is out of date, and the restaurant changes what they serve on a daily basis. The waitstaff and the chefs were able to accommodate our needs and decided to make the baked oysters and rockfish and chips that day. Ultimately, everyone was satisfied with their meals, and I am very glad that we visited this restaurant for our work celebratory lunch')

4 stars


In [36]:
test_model("I couldnt be more pleased after contacting Avi at The Klein Group. He was referred to me via my attorney, whose law firm has worked with him before. We had a need for some work regarding estate information gathering. He was quickly able to get to the bottom of our needs and assist us with solid results. Communications with him was always friendly, positive, respectful, and helpful. What more could we ask for? He's the private investigator I would turn to for future needs for sure, and Id recommend him to any friends or family for PI needs.")

5 stars


In [37]:
test_model('This Teaspoon location is located in the downtown Berkeley area. While this is a busier area, I easily found parking right in front of the shop. Ordering is through their kiosk by the counter, so not much service - even though there were only a few customers inside at the time. This is a smaller shop compared to their other shops I had been to. I ordered my usual from here, which is their Creamy Jasmine. I originally wanted boba and grass jelly for my drink, but for some reason, they do not have grass jelly at this location. Anyway, my drink was decent - it was a bit on the unsweetened / watered down side though. It was not as cold, creamy, or sweet as their other locations, even at the same ice / sweetness levels and with the same cream choice.')

4 stars


In [38]:
test_model('Was craving something sweet so decided to stop by. Butterfinger: 4.5/5 The frozen yogurt was super soft, almost kind of melty. I feel like it was more frozen solid in my previous visits but nonetheless, this one was still delicious! My absolute favorite is the peanut butter cup, give that a try!')

4 stars


In [39]:
test_model('Gadani is located in the Downtown Berkeley area - they have inconsistent hours, as I tried coming multiple times and the third time was the charm. Parking is kind of hit or miss here, as they are located in a busy area and on a busy street - I parked a couple of blocks away and walked over. This is a simple and small shop with very limited seating, but they do have a cute flower / leaf backdrop by the seating area, which is great for photos. No service whatsoever - ordering is through the kiosk by the front counter and the employee sitting by the counter does not speak to you / welcome you, nor does she even looks up from her phone. I ordered their Ice Cream Combo with your choice of egg waffle, ice cream flavor (1 flavor), and 2 complimentary toppings. For my egg waffle - I chose matcha, for my ice cream scoop - I chose matcha as well, and for my complimentary toppings - I chose condensed milk and whipped cream. Egg waffle was alright - a bit on the drier side with not much matcha taste to it, while the ice cream did not have a strong matcha taste at all. It was about $10, which I thought was a bit steep for what you get.')

4 stars


In [40]:
test_model("Very cute bakery with a minimalistic and clean aesthetic. I've had this place bookmarked for a while, and I can't remember if I've been here before or not. I do remember seeing long lines whenever I'm in Japan Town, which is usually on the weekends. Luckily, it wasn't busy when I came yesterday (Monday) afternoon. I got the Injeolmi Croissant and Garlic Chive Scone. The croissant had soy bean powder on top and Korean rice cake (tasted like mochi) inside. I thought the croissant was quite powdery. Because I didn't warm it up or eat it fresh, maybe it wasn't as good as it could've been or as good as I would've liked? Then again, I ate it almost immediately after I got home (about an hour or so later). On the other hand, I loved the scone. I just really love savory scones, especially when there's garlic and chive/spinach. The garlic chive flavor wasn't super strong. I wouldn't have minded if it were slightly stronger. Nonetheless, I would definitely get this again")

4 stars


In [41]:
test_model("This place gets PACKED on weekends, so I had to come on a Wednesday at an off time. Even so, plenty of people were there. We sat outside but it was a bit chilly, so we were offered cozy blankets to keep us warm -- so kind! I got the Abruzzo Lasagna, but everything looked so good that I don't think you could go wrong. It wonderfully zesty and crispy, and the cheese stretch was beautiful. Even the starter salad was great -- loved the thicker slices of tomato. Everyone there was so friendly and made us feel right at home. Just a great cozy vibe from this place, absolutely have to come here again.")

4 stars


In [ ]:
#new_model = tf.keras.models.load_model('star_system')